In [1]:
from src.DataLoader import gdfLoader,ReceiverLoader,RSSIDataLoader
from func.utils.getMarkerDictList import getMarkerDictList
from src.DataLoader import Room,Receiver

INFO - Logger configured


In [2]:
#Map Data
MapDataPath="data/cartePalaiseau.csv"
map_gdf=gdfLoader(MapDataPath)
#Receiver Data
ReceiverDataPath="data/carteModulePalaiseau.csv"
receiver_gdf=ReceiverLoader(ReceiverDataPath)

In [3]:
from src.DataLoader import room_dict_generator,room_r_tree_generator
room_dict=room_dict_generator(map_gdf)
room_r_tree=room_r_tree_generator(room_dict)


In [4]:
from src.DataLoader import build_receiver_dict
receiver_dict= build_receiver_dict(receiver_gdf,room_dict,room_r_tree)


In [5]:

from src.Symbolic_Localisation import ZoneSelector
Palaiseau_zone_selector=ZoneSelector(room_r_tree=room_r_tree,room_dict=room_dict)
        

In [6]:
from shapely import equals_exact
def test_case1(receiver_dict):
    receiver=receiver_dict['A8032A31204A']
    rssi=-40
    zone=Palaiseau_zone_selector.zone_selection(receiver,rssi)
    assert len(zone)==1,f'Error: point in more than one room'
    zone=zone[0]
    assert equals_exact(zone,receiver.room.cartesian_polygon),f'Error: {zone} should be {receiver.room.cartesian_polygon}'
test_case1(receiver_dict)
def test_full_treshold(receiver_dict):
    for receiver in receiver_dict.values():
        rssi=-40
        zone=Palaiseau_zone_selector.zone_selection(receiver,rssi)
        assert len(zone)==1,f'Error: point in more than one room'
        zone=zone[0]
        assert equals_exact(zone,receiver.room.cartesian_polygon),f'Error: {zone} should be {receiver.room.cartesian_polygon}'
test_full_treshold(receiver_dict)

In [7]:
from src.visualization import MapCreation

map_object=MapCreation(receiver_dict,room_dict)


In [8]:
import ast
import os
from src.DataLoader import gdfLoader,ReceiverLoader
from src import API
from src.preprocessing import Preprocessor
from src.preprocessing import remove_duplicates_Cleaner,moving_average_Filter,moving_max_Filter,moving_max_averaged_Filter,blank_Filter
import pandas as pd
from src.visualization import rssi_viewer

#API call
callApi=API()
start="2023-06-28 17:01:00.000000"
end="2023-06-28 17:11:10.000000"
mac_module=ast.literal_eval(os.environ['MAC_WEAR_DICT'])['3480']


raw_data=callApi.getRawDataForCartoWear(mac_module,pd.to_datetime(start),pd.to_datetime(end))
#Preprocessing
sampling_time=1
window='10S'
preprocessor=Preprocessor(raw_data,sampling_time=sampling_time)
rssi_df=preprocessor.set_cleaner(remove_duplicates_Cleaner())\
        .set_filter(moving_max_Filter("2s"))\
        .process()
fig=rssi_viewer(rssi_df)


INFO - data Cleaning


In [ ]:
rssi_df.dropna(inplace=True)
rssi_df

,timestamp,macModule,rssi
1,2023-06-28 15:01:01.000,A8032A311F96,-77.0
2,2023-06-28 15:01:01.000,A8032A311F6A,-61.0
4,2023-06-28 15:01:02.000,A8032A311F96,-77.0
5,2023-06-28 15:01:02.000,A8032A311F6A,-61.0
6,2023-06-28 15:01:02.500,A8032A311DAA,-72.0
...,...,...,...
4696,2023-06-28 15:11:09.000,A8032A31204A,-66.0
4697,2023-06-28 15:11:09.000,C45BBE37B346,-79.0
4698,2023-06-28 15:11:10.000,C45BBE37B346,-79.0
4699,2023-06-28 15:11:10.000,A8032A31204A,-66.0


In [ ]:
fig=rssi_viewer(rssi_df)


In [ ]:
import shapely

def add_element_to_list(list_dict,list_name, element):
    if list_name in list_dict:
        list_dict[list_name].append(element)
    else:
        list_dict[list_name] = [element]
def possible_room_union(zone_list:list)->shapely.geometry.Polygon:
    cartesian_polygon_list=[]
    for zone in zone_list:
        cartesian_polygon_list.append(zone)
    return(shapely.union_all(cartesian_polygon_list))
    

In [ ]:
#going through all the data and finding the possible rooms for each timestamp and for each receiver
all_zone={}
for row in rssi_df.iterrows():
    #init
    row=row[1]
    timestamp=row['timestamp']
    #all the possible room according to one receiver
    zone_list=Palaiseau_zone_selector.zone_selection(receiver_dict[row['macModule']],row['rssi'])
    #union of all the possible room according to one of the receiver
    possible_rooms=possible_room_union(zone_list)
    
    #adding this union to the dict with keys: timestamp
    add_element_to_list(all_zone,timestamp,possible_rooms)
res={}


for timestamp,instant_zone in all_zone.items():
    zone=shapely.intersection_all(instant_zone)
    if zone.is_empty:
        zone=shapely.union_all(instant_zone)
    res[timestamp]=zone
res


center=[244979.34863026388, 6226335.888478451] radius=5
244984.34863026388 6226335.888478451
244981.84863026388 6226340.21860547
244976.84863026388 6226340.21860547
244974.34863026388 6226335.888478451
244976.84863026388 6226331.558351433
244981.84863026388 6226331.558351433
center=[244974.01139043097, 6226332.49732344] radius=2.5
244976.51139043097 6226332.49732344
244975.26139043097 6226334.66238695
244972.76139043097 6226334.66238695
244971.51139043097 6226332.49732344
244972.76139043097 6226330.332259931
244975.26139043097 6226330.332259931
center=[244979.34863026388, 6226335.888478451] radius=5
244984.34863026388 6226335.888478451
244981.84863026388 6226340.21860547
244976.84863026388 6226340.21860547
244974.34863026388 6226335.888478451
244976.84863026388 6226331.558351433
244981.84863026388 6226331.558351433
center=[244974.01139043097, 6226332.49732344] radius=2.5
244976.51139043097 6226332.49732344
244975.26139043097 6226334.66238695
244972.76139043097 6226334.66238695
244971.5

/home/gchenu/Documents/.venv/lib/python3.10/site-packages/shapely/set_operations.py:178: RuntimeWarning:

invalid value encountered in intersection_all



{Timestamp('2023-06-28 15:01:01'): <POLYGON ((244967.949 6226327.486, 244969.199 6226334.021, 244969.628 622633...>,
 Timestamp('2023-06-28 15:01:02'): <POLYGON ((244967.949 6226327.486, 244969.199 6226334.021, 244969.628 622633...>,
 Timestamp('2023-06-28 15:01:02.500000'): <POLYGON ((244976.676 6226332.749, 244975.562 6226326.217, 244967.949 622632...>,
 Timestamp('2023-06-28 15:01:03'): <POLYGON ((244967.949 6226327.486, 244969.199 6226334.021, 244969.628 622633...>,
 Timestamp('2023-06-28 15:01:03.500000'): <POLYGON ((244976.676 6226332.749, 244975.562 6226326.217, 244967.949 622632...>,
 Timestamp('2023-06-28 15:01:04'): <POLYGON ((244983.885 6226324.836, 244979.855 6226325.396, 244975.562 622632...>,
 Timestamp('2023-06-28 15:01:04.500000'): <POLYGON ((244976.676 6226332.749, 244975.562 6226326.217, 244967.949 622632...>,
 Timestamp('2023-06-28 15:01:05'): <POLYGON ((244983.885 6226324.836, 244979.855 6226325.396, 244975.562 622632...>,
 Timestamp('2023-06-28 15:01:07'): <POLYGON

transfer the coordinates of the polygon obtained at each timestamp to lat long for plotting on the map

In [ ]:
from pyproj import Transformer
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
def polygon_to_lat_lon(polygon:shapely.geometry.Polygon)->shapely.geometry.Polygon:
    """
    Return the room in which the point is located.
    
    """
    coords=polygon.exterior.coords
    point_list=[transformer.transform(x,y) for x,y in coords] 
    polygon=shapely.geometry.Polygon(point_list)
    return(polygon)
res_in_lat_long={}
for timestamp,polygon in res.items():
    if isinstance(polygon,shapely.geometry.MultiPolygon):
        polygon_list=[]
        for poly in polygon.geoms:
            # on enleve les elements qui ne sont pas des polygones (line string,multi line string,point)
            if isinstance(poly, shapely.geometry.Polygon):
                polygon_list.append(polygon_to_lat_lon(poly))
        if polygon_list:
            res_in_lat_long[timestamp]=shapely.geometry.MultiPolygon(polygon_list)
        
    elif isinstance(polygon,shapely.geometry.Polygon) :
        res_in_lat_long[timestamp]=polygon_to_lat_lon(polygon)
res_in_lat_long

{Timestamp('2023-06-28 15:01:01'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-06-28 15:01:02'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-06-28 15:01:02.500000'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-06-28 15:01:03'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-06-28 15:01:03.500000'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-06-28 15:01:04'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-06-28 15:01:04.500000'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-06-28 15:01:05'): <POLYGON ((48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2.201, 48.713 2....>,
 Timestamp('2023-06-28 15:01:07'): <POLYGON

In [ ]:
from  src.visualization import timed_room_Feature
from folium.plugins import TimestampedGeoJson

polygonal_features=timed_room_Feature(res_in_lat_long)

In [ ]:
TimestampedGeoJson({"type": "FeatureCollection","features": polygonal_features},
                    period='PT1S',
                    duration='PT1S',
                    transition_time=0,  
                    auto_play=True,date_options='YYYY/MM/DD HH:mm:ss:SSS',
                    ).add_to(map_object)

map_object

In [ ]:
map_object.save("output/wip.html")


In [ ]:
import folium
## add this circle to map :
circle=discrete_circle(6,(48,2),2.5)

geojson={
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [[x,y] for x,y in circle.exterior.coords]
            ]
        ]
    }
}
print(geojson)
folium.GeoJson(geojson, name="geojson").add_to(map_object)
map_object


center=(48, 2) radius=2.5
50.5 2.0
49.25 4.165063509461096
46.75 4.165063509461097
45.5 2.0000000000000004
46.75 -0.16506350946109594
49.25 -0.16506350946109638
{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[[50.5, 2.0], [49.25, 4.165063509461096], [46.75, 4.165063509461097], [45.5, 2.0000000000000004], [46.75, -0.16506350946109594], [49.25, -0.16506350946109638], [50.5, 2.0]]]]}}
